In [3]:
import pandas as pd 
import wandb
from tqdm import tqdm
import requests
import os
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import metrics
import matplotlib.pyplot as plt
import scipy.stats as stats
import matplotlib.colors as mcolors

In [4]:
from read_wandb import wandb_results
api = wandb_results("NLP2024_PROJECT_206320772_313510679", wandb_username="shalom_and_amit")

BASE_METRIC = "accuracy_per_mean_user_and_bot"


In [5]:
def result_metric(sweeps, group_name, drop_list=[0], drop_HPT=False, metric=BASE_METRIC, epoch="best", flag=False):
    df = api.get_sweeps_results(sweeps, metric=metric)
    config_cols = [c for c in df.columns if "config_" in c and c!="config_wandb_run_id" and c!="config_online_simulation_size"]
    HPT_cols = [col for col in config_cols if df[col].nunique() > 1]
    if drop_HPT:
        df=df.drop([c for c in HPT_cols if not c in ["config_LLM_SIM_SIZE", "config_seed"]], axis=1)
        HPT_cols = ["config_LLM_SIM_SIZE", "config_seed"]
    numeric_cols = df.select_dtypes(include=np.number).columns
    if flag:
      numeric_cols = df.select_dtypes(include=np.number).columns.to_list()
      numeric_cols.append("config_features")

    df_numeric = df[numeric_cols]

    grouped = df_numeric.groupby([c for c in HPT_cols if c != "config_seed" and c != "config_FEATURES_PATH" and c != "config_input_dim" and c !=	"config_REVIEW_DIM"])

    mean_df = grouped.mean()
    std_df = grouped.std()
    for col in config_cols:
        if col not in mean_df.columns:
            mean_df[col] = df[col]

    if epoch=="best":
        best_col = mean_df[[c for c in mean_df.columns if (metric in c and metric[-4:] == c.split("_epoch")[0][-4:])]].idxmax(axis=1)
    else:
        best_col = mean_df[[c for c in mean_df.columns if f"{metric}_epoch{epoch}" in c]].idxmax(axis=1)
    result = grouped.apply(lambda x: x[best_col.loc[x.name]].values)
    means = grouped.apply(lambda x: x[best_col.loc[x.name]].mean())
    stds = grouped.apply(lambda x: x[best_col.loc[x.name]].std())


    df_cols = {'mean': means, 'std': stds, 'values': result.values}
    if epoch == "best": df_cols['epoch'] = best_col.apply(lambda x: int(x.split("epoch")[1]) if "epoch" in x else "last")

    df_cols['CI'] = result.apply(lambda x: bootstrap_ci(x))
    summary_df = pd.DataFrame(df_cols, index=best_col.index)
    for d in drop_list:
        if d in summary_df.index:
            summary_df=summary_df.drop(d)
    if len(summary_df.index.names) == 1:
        return summary_df.rename_axis(group_name)
    else:
        return summary_df
    
    
def bootstrap_ci(data, n_bootstrap=1000, ci=0.95):
    bootstrapped_means = []
    for _ in range(n_bootstrap):
        sample = np.random.choice(data, size=len(data), replace=True)
        bootstrapped_means.append(np.mean(sample))
    lower_bound = np.percentile(bootstrapped_means, (1 - ci) / 2 * 100)
    upper_bound = np.percentile(bootstrapped_means, (1 + ci) / 2 * 100)
    return lower_bound, upper_bound


In [6]:
sweep_results = result_metric(["4wkmpa6k"], "LLMs", drop_HPT=False, epoch="best", flag=True)
sweep_results

Total number of sweeps: 1
Download sweep_id='4wkmpa6k' data...


mean       std  \
config_features config_online_simulation_factor                       
BERT            0                                0.826854  0.002723   
                4                                0.831485  0.003222   
EFs             0                                0.828898  0.001633   
                4                                0.836871  0.002149   
GPT4            0                                0.833291  0.002611   
                4                                0.832523  0.002281   

                                                                                            values  \
config_features config_online_simulation_factor                                                      
BERT            0                                [0.8240678419478681, 0.8313392411637519, 0.826...   
                4                                [0.8346011801111206, 0.8322325171622887, 0.826...   
EFs             0                                [0.8277120396253037, 0.8267503772423906, 0.829...   
                4                                [0.8396486019565295, 0.8347056199456093, 0.834...   
GPT4            0                                [0.830059882220964, 0.8358669977445071, 0.8350...   
                4                                [0.8316417175518412, 0.8317282817060091, 0.832...   

                                                 epoch  \
config_features config_online_simulation_factor          
BERT            0                                    9   
                4                                   17   
EFs             0                                    8   
                4                                   11   
GPT4            0                                    5   
                4                                   18   

                                                                                       CI  
config_features config_online_simulation_factor                                            
BERT            0                                (0.8249102973443584, 0.8292731368738888)  
                4                                 (0.828573492745374, 0.8336523661153443)  
EFs             0                                 (0.827573664846432, 0.8301357228098227)  
                4                                  (0.83526548424105, 0.8384826591010572)  
GPT4            0                                 (0.8312994008163168, 0.835316962825928)  
                4                                (0.8310187355234978, 0.8344177175690349)

# For HyperParameterTuning

For every configuration that you test in the sweep, you will receive in the table the average, standard deviation, all the values obtained for the different seed values, and also the confidence interval within which the result is located at a confidence level of 95%.

When epoch="best" is defined, you can check in which epoch the best result is obtained. If epoch=5 is defined, you will receive the result obtained for epoch number 5.

You can test multiple sweeps simultaneously by entering them into the list found in the first element of the function result_metric.

In [8]:
sweep_results = result_metric(["4wkmpa6k"], "LLMs", drop_HPT=False, epoch="best", flag=True)
sweep_results

Total number of sweeps: 1
Download sweep_id='4wkmpa6k' data...


mean       std  \
config_features config_online_simulation_factor                       
BERT            0                                0.826854  0.002723   
                4                                0.831485  0.003222   
EFs             0                                0.828898  0.001633   
                4                                0.836871  0.002149   
GPT4            0                                0.833291  0.002611   
                4                                0.832523  0.002281   

                                                                                            values  \
config_features config_online_simulation_factor                                                      
BERT            0                                [0.8240678419478681, 0.8313392411637519, 0.826...   
                4                                [0.8346011801111206, 0.8322325171622887, 0.826...   
EFs             0                                [0.8277120396253037, 0.8267503772423906, 0.829...   
                4                                [0.8396486019565295, 0.8347056199456093, 0.834...   
GPT4            0                                [0.830059882220964, 0.8358669977445071, 0.8350...   
                4                                [0.8316417175518412, 0.8317282817060091, 0.832...   

                                                 epoch  \
config_features config_online_simulation_factor          
BERT            0                                    9   
                4                                   17   
EFs             0                                    8   
                4                                   11   
GPT4            0                                    5   
                4                                   18   

                                                                                       CI  
config_features config_online_simulation_factor                                            
BERT            0                                (0.8250988975088474, 0.8290362120691195)  
                4                                (0.8285721776495365, 0.8336523661153443)  
EFs             0                                (0.8276589383208109, 0.8301356815345793)  
                4                                 (0.8352607151670718, 0.838481360180821)  
GPT4            0                                (0.8312994008163168, 0.8351621494456666)  
                4                                (0.8310014226926642, 0.8344177175690349)

# Result for a specific epoch

In [8]:
sweep_results = result_metric(["kb9be58j"], "LLMs", drop_HPT=False, epoch=10)
sweep_results

Total number of sweeps: 1
Download sweep_id='kb9be58j' data...
['config_seed', 'config_features', 'config_input_dim', 'config_REVIEW_DIM', 'config_FEATURES_PATH', 'config_online_simulation_factor']


mean  \
config_features config_input_dim config_REVIEW_DIM config_FEATURES_PATH  config_online_simulation_factor             
BERT            49               36                data/BERT_PCA_36.csv  0                                0.791324   
                                                                         4                                0.787746   
EFs             50               37                data/EFs_by_GPT35.csv 0                                0.795179   
                                                                         4                                0.804494   
GPT4            49               36                data/GPT4_PCA_36.csv  0                                0.792382   
                                                                         4                                0.789859   

                                                                                                               std  \
config_features config_input_dim config_REVIEW_DIM config_FEATURES_PATH  config_online_simulation_factor             
BERT            49               36                data/BERT_PCA_36.csv  0                                0.004130   
                                                                         4                                0.002023   
EFs             50               37                data/EFs_by_GPT35.csv 0                                0.004638   
                                                                         4                                0.004319   
GPT4            49               36                data/GPT4_PCA_36.csv  0                                0.006579   
                                                                         4                                0.001764   

                                                                                                                                                     values  \
config_features config_input_dim config_REVIEW_DIM config_FEATURES_PATH  config_online_simulation_factor                                                      
BERT            49               36                data/BERT_PCA_36.csv  0                                [0.7913288142079798, 0.7964738446311359, 0.794...   
                                                                         4                                [0.7853011998466384, 0.7880448817556057, 0.790...   
EFs             50               37                data/EFs_by_GPT35.csv 0                                [0.788716651726378, 0.7969832115134915, 0.7940...   
                                                                         4                                [0.8113387801279366, 0.8016887599672099, 0.802...   
GPT4            49               36                data/GPT4_PCA_36.csv  0                                [0.7926385318762957, 0.8021579175889718, 0.783...   
                                                                         4                                [0.7906707039977815, 0.7882686273661637, 0.789...   

                                                                                                                                                CI  
config_features config_input_dim config_REVIEW_DIM config_FEATURES_PATH  config_online_simulation_factor                                            
BERT            49               36                data/BERT_PCA_36.csv  0                                (0.7883271049308528, 0.7945211780625762)  
                                                                         4                                (0.7861906690533982, 0.7893304571102691)  
EFs             50               37                data/EFs_by_GPT35.csv 0                                (0.7914422896969608, 0.7990928008535583)  
                                                                         4                                  (0.8015579276967888, 0.80837621364379)  
GPT4            49               36                da